In [10]:
from google.colab import files
uploaded = files.upload()

Saving emoji_dataset.zip to emoji_dataset (1).zip


In [11]:
import zipfile

with zipfile.ZipFile("emoji_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("emoji_dataset")


In [23]:
import tensorflow as tf

BATCH_SIZE = 32
IMG_SIZE = (128, 128)

dataset = tf.keras.utils.image_dataset_from_directory(
    "emoji_dataset/dataset",
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42
)

batch_count = 0
for batch in dataset:
    print(batch[0].shape)
    batch_count += 1

print("Total batches:", batch_count)

Found 540 files belonging to 18 classes.
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(28, 128, 128, 3)
Total batches: 17


In [25]:
import os
print(os.listdir("emoji_dataset/dataset"))



['cloud', 'flushed-face', 'smiling-sunglasses', 'savoring-food', 'smirking-face', 'pouting-face', 'raised-eyebrow', 'grinning-squinting', 'beaming-face', 'heart', 'grinning-face', 'tears-of-joy', 'smiling-heart', 'relieved-face', 'smiling-tear', 'grimacing-face', 'smiling-horns', 'face-spiral']


In [104]:
import tensorflow as tf

total_batches = tf.data.experimental.cardinality(dataset).numpy()

train_size = int(0.7 * total_batches)

train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size)

In [105]:
import collections

train_labels = []

for images, labels in train_ds:
    train_labels.extend(labels.numpy())

train_counts = collections.Counter(train_labels)

print("Train Dataset Label Distribution:")
for label, count in sorted(train_counts.items()):
    print(f"Label {label}: {count} images")


Train Dataset Label Distribution:
Label 0: 13 images
Label 1: 18 images
Label 2: 23 images
Label 3: 22 images
Label 4: 21 images
Label 5: 17 images
Label 6: 21 images
Label 7: 17 images
Label 8: 20 images
Label 9: 22 images
Label 10: 20 images
Label 11: 22 images
Label 12: 19 images
Label 13: 22 images
Label 14: 23 images
Label 15: 12 images
Label 16: 18 images
Label 17: 22 images


In [106]:
val_labels = []

for images, labels in val_ds:
    val_labels.extend(labels.numpy())

val_counts = collections.Counter(val_labels)

print("Validation Dataset Label Distribution:")
for label, count in sorted(val_counts.items()):
    print(f"Label {label}: {count} images")


Validation Dataset Label Distribution:
Label 0: 11 images
Label 1: 10 images
Label 2: 8 images
Label 3: 14 images
Label 4: 11 images
Label 5: 7 images
Label 6: 12 images
Label 7: 12 images
Label 8: 10 images
Label 9: 7 images
Label 10: 11 images
Label 11: 13 images
Label 12: 11 images
Label 13: 13 images
Label 14: 8 images
Label 15: 11 images
Label 16: 9 images
Label 17: 10 images


In [107]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1)
])

In [108]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    data_augmentation,

    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(len(dataset.class_names), activation='softmax')
])


In [109]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [111]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 0.7299 - loss: 0.8490 - val_accuracy: 0.7553 - val_loss: 0.7296
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.7201 - loss: 0.8270 - val_accuracy: 0.7766 - val_loss: 0.6432
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.7477 - loss: 0.7470 - val_accuracy: 0.7713 - val_loss: 0.7497
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.8212 - loss: 0.6568 - val_accuracy: 0.7872 - val_loss: 0.6356
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 174ms/step - accuracy: 0.7847 - loss: 0.6572 - val_accuracy: 0.7766 - val_loss: 0.6285
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.7782 - loss: 0.7282 - val_accuracy: 0.8245 - val_loss: 0.5961
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.8423 - loss: 0.6063 - val_accuracy: 0.8298 - val_loss: 0.6054
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.8158 - loss: 0.6632 - val_accuracy: 0.

In [112]:
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Loss: {loss:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9336 - loss: 0.3148
Validation Accuracy: 0.9309
Validation Loss: 0.2609


In [62]:
from google.colab import files
uploaded = files.upload()

Saving sunglass.jpg to sunglass.jpg


In [65]:
from PIL import ImageOps

In [66]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

IMG_SIZE=(128,128)

img_path = input("Enter the path to your PNG file: ")

img = Image.open(img_path).convert("L")
img = ImageOps.invert(img)
img = img.resize(IMG_SIZE)
img_array = np.stack((img,)*3, axis=-1)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

class_names = dataset.class_names
predicted_class_name = class_names[predicted_class_index]

print(f"Prediction: {predicted_class_name}")

Enter the path to your PNG file: sunglass.jpg
Prediction: heart
